## Урок 2. Загрузка данных и построение обучающей выборки. Анализ и предобработка датасета. Балансировка классов

Реальная задача машинного обучения - это история, прежде всего, про грамотно выстроенные процессы, которые покрывают полный цикл от сбора исходных данных до измеримого бизнес-эффекта. Последовательные преобразования данных и дальнейшее обучение модели на них объединят в процесс, называемый "пайплайном" (pipeline). В рамках курса предлагается собрать пайплайн, который будет заключаться в сборе исходных данных, их подготовке к обучению и обучение модели, то есть на входе пайплайна - сырые данные, на выходе - готовая модель. На практике пайплайн часто создают с использованием фреймворка [Apache Spark](https://ru.wikipedia.org/wiki/Apache_Spark).

Первый и, пожалуй, самый важный этап пайплайн - сбор данных, специфичен тем, что на нем происходит взаимодействие, как правило, между сервером, обрабатывающим данные (это может быть сервер Jupyter), и хранилищем данных. Особенность заключается в том, что при построении пайплайна нужно учитывать особенности такого взаимодействия. Хранилище может быть недоступно из-за каких-то сетевых проблем, в процессе загрузки может возникнуть какая-то ошибка, в конце концов можно превысить лимиты запрашиваемого объеда данных. Потенциальных проблем может быть много, и хорошо написанный пайплайн должен учитывать, если и не все, то уж точно большинство, возможных исключений и ошибок.

### Особенности загрузки данных

Рассмотрим на примере загрузки данных с помощью Apache Impala, каким образом можно контролировать загрузку данных и обрабатывать нестандартные ситуации. Для этого обратимся к такому инструменту Python, как __декоратор__.

Напомним, что функции в Python являются объектами, соответственно, их можно возвращать из другой функции или передавать в качестве аргумента. Также следует помнить, что функция в Python может быть определена и внутри другой функции. Декораторы — это, по сути, "обёртки", которые дают возможность изменить поведение функции, не изменяя её код.

Проиллюстрировать смысл декоратора можно следующим кодом:

In [1]:
import time
from datetime import datetime, timedelta
import pandas as pd

def time_format(sec):
    return str(timedelta(seconds=sec))

def my_decorator(function_to_decorate):
    # Внутри себя декоратор определяет функцию-"обёртку". Она будет обёрнута вокруг декорируемой,
    # получая возможность исполнять произвольный код до и после неё.
    def the_wrapper_around_the_original_function():
        print("Я - код, который отработает до вызова функции")
        function_to_decorate() # Вызов самой функции
        print("А я - код, срабатывающий после")
    # Вернём эту функцию
    return the_wrapper_around_the_original_function

Посмотрим, как можно на практике использовать декоратор. Напишем декоратор, который будет измерять время работы функции, наподобии __%%time__.

In [2]:
def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print('Время выполнения: {} секунды'.format(end-start))
        return return_value
    return wrapper

@time_it
def dummy_func(seconds_to_wait):
    time.sleep(seconds_to_wait)
    val = 'Функция спала ~{} секунды'.format(seconds_to_wait)
    return val

dummy = dummy_func(3)
print(dummy)

Время выполнения: 3.0030934810638428 секунды
Функция спала ~3 секунды


Итак, применим теперь декораторы к процессу загрузки данных. Напишем следующий декоратор:

In [3]:
def etl_loop(func):
    def _func(*args, **kwargs):
        _max_iter_cnt = 100
        for i in range(_max_iter_cnt):
            try:
                start_t = time.time()
                res = func(*args, **kwargs)
                run_time = time_format(time.time() - start_t)
                print('Run time "{}": {}'.format(func.__name__, run_time))
                return res
            except Exception as er:
                run_time = time_format(time.time() - start_t)
                print('Run time "{}": {}'.format(func.__name__, run_time))
                print('-'*50)
                print(er, '''Try № {}'''.format(i + 1))
                print('-'*50)
        raise Exception('Max error limit exceeded: {}'.format(_max_iter_cnt))
    return _func

Декоратор говорит нам о том, что декорируемая функция будет выполняться максимум 100 раз, будет отлеживаться время выполнения, в случае ошибки - выводим текст ошибки и номер попытки перезапуска функции, а в случае достижения лимита по итерациям останавливаем выполнение кода.

Далее, приведем примеры нескольких функций, которые загружают данные на сервер Jupyter и "обернуты" декораторами:

In [4]:
@etl_loop
def loading_step(**kwargs):
    with ETL() as etl:
        tbl = kwargs['table']
        print('Loading -> {}...'.format(tbl))
        etl.create_table(sql=getattr(etl, 'CREATE_{}_SQL'.format(tbl.upper())))
        etl.insert_table(sql=getattr(etl, '{}_DATA_SQL'.format(tbl.upper())), 
                         min_level=kwargs['min_level'],
                         max_level=kwargs['max_level'], 
                         churned_start_date=kwargs['churned_start_date'], 
                         churned_end_date=kwargs['churned_end_date'],
                         data_start_date=kwargs['data_start_date'], 
                         data_end_date=kwargs['data_end_date'])
        etl.insert_table(sql=etl.CHECK_SQL, 
                         tbl_name='usr_erin.churn_{}'.format(tbl), 
                         is_res_needed=True)
        print('Rows = {}, Users = {}'.format(etl.res[0][0], etl.res[0][1]))

@etl_loop
def save_files(table, path):
    with ETL() as etl:
        print('Saving "{}" into *.csv...'.format(table))
        sql_to_file('select * from usr_erin.churn_{}'.format(table), 
                    '{}{}.csv'.format(path, 'churn_{}'.format(table)))
        
@etl_loop
def load_data(min_level='1',
              max_level='100',
              churned_start_date='2019-01-01', 
              churned_end_date=datetime.strftime(datetime.now()-timedelta(days=30), '%Y-%m-%d'),
              data_start_date='2019-01-01',
              data_end_date=datetime.strftime(datetime.now()-timedelta(days=30), '%Y-%m-%d'), 
              save_to_csv=True, 
              start_with='sample',
              raw_data_path='../input/'):
    
    sources = ['sample', 'profiles', 'payments', 'reports', 'abusers', 'logins', 'pings', 'sessions', 'shop']
    start_index = sources.index(start_with)
    
    for tbl in sources[start_index:]:
        loading_step(table=tbl, 
                     min_level=min_level, 
                     max_level=max_level, 
                     churned_start_date=churned_start_date, 
                     churned_end_date=churned_end_date, 
                     data_start_date=data_start_date, 
                     data_end_date=data_end_date)
        if save_to_csv:
            save_files(table=tbl, 
                       path=raw_data_path)
        
    print('Done! All data has been loaded!')

Видим, что данные загружаются поэтапно, источник за источником, при этом при загрузке каждого из компонентов соединение с Impala инициализируется заново и после отработки каждого из запросов данные сохраняются в csv файл (подумайте, зачем может быть нужна такая поэтапность?).

### Создание датасета

Ознакомившись с тонкостями загрузки данных, приступим к созданию датасета из сырых данных, которые были определены на предыдущем уроке. Напишем код, который будет из сырых данных собирать датасет.

Напомним, что у нас в распоряжении есть данные за последние 4 недели жизни игрока в проекте, и на основании каждого из "динамических" индикаторов мы будем строить по 4 признака, которые будут соответствовать каждой из 4 недель последнего месяца жизни игрока в проекте. Данные дополнительно не будем пока обрабатывать.

In [5]:
def build_dataset_raw(churned_start_date='2019-01-01', 
                      churned_end_date='2019-02-01', 
                      inter_list=[(1,7),(8,14)],
                      raw_data_path='train/',
                      dataset_path='dataset/', 
                      mode='train'):
    
    start_t = time.time()
 
    sample = pd.read_csv('{}sample.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    profiles = pd.read_csv('{}profiles.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    payments = pd.read_csv('{}payments.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    reports = pd.read_csv('{}reports.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    abusers = pd.read_csv('{}abusers.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    logins = pd.read_csv('{}logins.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    pings = pd.read_csv('{}pings.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    sessions = pd.read_csv('{}sessions.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    shop = pd.read_csv('{}shop.csv'.format(raw_data_path), sep=';', na_values=['\\N', 'None'], encoding='utf-8')
    
    print('Run time (reading csv files): {}'.format(time_format(time.time()-start_t)))    
#-----------------------------------------------------------------------------------------------------    
    print('NO dealing with outliers, missing values and categorical features...')
#-----------------------------------------------------------------------------------------------------        
    # На основании дня отвала (last_login_dt) строим признаки, которые описывают активность игрока перед уходом
    
    print('Creating dataset...')
    # Создадим пустой датасет - в зависимости от режима построения датасета - train или test
    if mode == 'train':
        dataset = sample.copy()[['user_id', 'is_churned', 'level', 'donate_total']]
    elif mode == 'test':
        dataset = sample.copy()[['user_id', 'level', 'donate_total']]

    # Пройдемся по всем источникам, содержащим "динамичекие" данные
    for df in [payments, reports, abusers, logins, pings, sessions, shop]:

        # Получим 'day_num_before_churn' для каждого из значений в источнике для определения недели
        data = pd.merge(sample[['user_id', 'login_last_dt']], df, on='user_id')
        data['day_num_before_churn'] = 1 + (data['login_last_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d')) - 
                                data['log_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))).apply(lambda x: x.days)
        df_features = data[['user_id']].drop_duplicates().reset_index(drop=True)

        # Для каждого признака создадим признаки для каждого из времененно интервала (в нашем примере 4 интервала по 7 дней)
        features = list(set(data.columns) - set(['user_id', 'login_last_dt', 'log_dt', 'day_num_before_churn']))
        print('Processing with features:', features)
        for feature in features:
            for i, inter in enumerate(inter_list):
                inter_df = data.loc[data['day_num_before_churn'].between(inter[0], inter[1], inclusive=True)].\
                                groupby('user_id')[feature].mean().reset_index().\
                                rename(index=str, columns={feature: feature+'_{}'.format(i+1)})
                df_features = pd.merge(df_features, inter_df, how='left', on='user_id')

        # Добавляем построенные признаки в датасет
        dataset = pd.merge(dataset, df_features, how='left', on='user_id')
        
        print('Run time (calculating features): {}'.format(time_format(time.time()-start_t)))

    # Добавляем "статические" признаки
    dataset = pd.merge(dataset, profiles, on='user_id')
#---------------------------------------------------------------------------------------------------------------------------
    dataset.to_csv('{}dataset_raw_{}.csv'.format(dataset_path, mode), sep=';', index=False)
    print('Dataset is successfully built and saved to {}, run time "build_dataset_raw": {}'.\
          format(dataset_path, time_format(time.time()-start_t)))

Применим написанную функцию к нашим данным, при этом построим датасет отдельно для трейна и теста:

In [6]:
# Следует из исходных данных
CHURNED_START_DATE = '2019-09-01' 
CHURNED_END_DATE = '2019-10-01'

INTER_1 = (1,7)
INTER_2 = (8,14)
INTER_3 = (15,21)
INTER_4 = (22,28)
INTER_LIST = [INTER_1, INTER_2, INTER_3, INTER_4]

In [7]:
build_dataset_raw(churned_start_date=CHURNED_START_DATE,
                  churned_end_date=CHURNED_END_DATE,
                  inter_list=INTER_LIST,
                  raw_data_path='train/',
                  dataset_path='dataset/', 
                  mode='train')

Run time (reading csv files): 0:01:23.901281
NO dealing with outliers, missing values and categorical features...
Creating dataset...
Processing with features: ['trans_amt', 'pay_amt']
Run time (calculating features): 0:01:51.854116
Processing with features: ['reports_amt']
Run time (calculating features): 0:03:22.588922
Processing with features: ['sess_with_abusers_amt']
Run time (calculating features): 0:07:53.086243
Processing with features: ['disconnect_amt', 'session_amt']
Run time (calculating features): 0:13:20.591053
Processing with features: ['avg_min_ping']
Run time (calculating features): 0:18:35.709343
Processing with features: ['session_player', 'win_rate', 'leavings_rate', 'kd']
Run time (calculating features): 0:23:59.450460
Processing with features: ['silver_spent', 'gold_spent']
Run time (calculating features): 0:29:20.988729
Dataset is successfully built and saved to dataset/, run time "build_dataset_raw": 0:29:48.019019


In [8]:
build_dataset_raw(churned_start_date=CHURNED_START_DATE,
                  churned_end_date=CHURNED_END_DATE,
                  inter_list=INTER_LIST,
                  raw_data_path='test/',
                  dataset_path='dataset/', 
                  mode='test')

Run time (reading csv files): 0:00:05.964428
NO dealing with outliers, missing values and categorical features...
Creating dataset...
Processing with features: ['trans_amt', 'pay_amt']
Run time (calculating features): 0:00:08.178363
Processing with features: ['reports_amt']
Run time (calculating features): 0:00:15.600667
Processing with features: ['sess_with_abusers_amt']
Run time (calculating features): 0:00:40.310513
Processing with features: ['disconnect_amt', 'session_amt']
Run time (calculating features): 0:01:08.838487
Processing with features: ['avg_min_ping']
Run time (calculating features): 0:01:36.332214
Processing with features: ['session_player', 'win_rate', 'leavings_rate', 'kd']
Run time (calculating features): 0:02:03.799303
Processing with features: ['silver_spent', 'gold_spent']
Run time (calculating features): 0:02:33.566338
Dataset is successfully built and saved to dataset/, run time "build_dataset_raw": 0:02:35.930178


In [9]:
train = pd.read_csv('dataset/dataset_raw_train.csv', sep=';')
test = pd.read_csv('dataset/dataset_raw_test.csv', sep=';')
print(train.shape, test.shape)

(469475, 62) (44764, 61)


In [10]:
train.head()

,user_id,is_churned,level,donate_total,trans_amt_1,trans_amt_2,trans_amt_3,trans_amt_4,pay_amt_1,pay_amt_2,...,gold_spent_1,gold_spent_2,gold_spent_3,gold_spent_4,age,gender,days_between_reg_fl,days_between_fl_df,has_return_date,has_phone_number
0,1e7edd8347e3aaeedf8c494b11240851e3fa0ad231b8f8...,0,43,88730.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,78.666667,0.000000,26.0,M,0,7,1,1
1,f43cac5f14e06ca039b173e14c323ac0c1fd8492f0cf08...,0,50,44149.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.000000,0.000000,27.0,M,0,37,1,1
2,cc7450e0b182947998534ef137b05e07109c100aced0b6...,0,37,44931.00000,1.0,1.0,NaN,2.0,63.000000,350.0,...,104.285714,0.0,1.428571,2.857143,21.0,M,0,153,1,1
3,5c583d57a1e9e53341fc239d41fb6983e667a04b1b4d94...,0,20,37538.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,22.0,M,0,156,1,1
4,9bbaa1a2501e8dc83cf6c0c54ef139c75c99de09dcf4dc...,0,10,4100.97998,1.0,NaN,NaN,NaN,66.580002,NaN,...,0.000000,0.0,0.000000,0.000000,2.0,M,0,21,1,1


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469475 entries, 0 to 469474
Data columns (total 62 columns):
user_id                    469475 non-null object
is_churned                 469475 non-null int64
level                      469475 non-null int64
donate_total               469475 non-null float64
trans_amt_1                67485 non-null float64
trans_amt_2                60928 non-null float64
trans_amt_3                56720 non-null float64
trans_amt_4                57896 non-null float64
pay_amt_1                  67485 non-null float64
pay_amt_2                  60928 non-null float64
pay_amt_3                  56720 non-null float64
pay_amt_4                  57896 non-null float64
reports_amt_1              144916 non-null float64
reports_amt_2              145909 non-null float64
reports_amt_3              147492 non-null float64
reports_amt_4              147503 non-null float64
sess_with_abusers_amt_1    261313 non-null float64
sess_with_abusers_amt_2    259432 n

### Обработка датасета

Каждый игрок теперь описывается множеством из N признаков, но в датасете есть пропуски, часть из которых обусловлены пробелами в исходных сырых данных, а часть отсутствует из-за неактивности игрока в тот или иной период времени, поэтому теперь обработаем датасет, чтобы убрать пропуски. Также обработаем явные выбросы и категориальные признаки. Напишем соответствующую функцию, которая делает это.

__Важный момент:__ вам предлагается попробовать обработать данные самостоятельно, итоговый препроцессинг зависит только от вас, здесь указан лишь пример.

In [12]:
def prepare_dataset(dataset, 
                    dataset_type='train',
                    dataset_path='dataset/'):
    print(dataset_type)
    start_t = time.time()
    print('Dealing with missing values, outliers, categorical features...')
    
    # Профили
    dataset['age'] = dataset['age'].fillna(dataset['age'].median())
    dataset['gender'] = dataset['gender'].fillna(dataset['gender'].mode()[0])
    dataset.loc[~dataset['gender'].isin(['M', 'F']), 'gender'] = dataset['gender'].mode()[0]
    dataset['gender'] = dataset['gender'].map({'M': 1., 'F':0.})
    dataset.loc[(dataset['age'] > 80) | (dataset['age'] < 7), 'age'] = round(dataset['age'].median())
    dataset.loc[dataset['days_between_fl_df'] < -1, 'days_between_fl_df'] = -1
    # Пинги
    for period in range(1,len(INTER_LIST)+1):
        col = 'avg_min_ping_{}'.format(period)
        dataset.loc[(dataset[col] < 0) | 
                    (dataset[col].isnull()), col] = dataset.loc[dataset[col] >= 0][col].median()
    # Сессии и прочее
    dataset.fillna(0, inplace=True)
    dataset.to_csv('{}dataset_{}.csv'.format(dataset_path, dataset_type), sep=';', index=False)
         
    print('Dataset is successfully prepared and saved to {}, run time (dealing with bad values): {}'.\
          format(dataset_path, time_format(time.time()-start_t))) 

In [13]:
prepare_dataset(dataset=train, dataset_type='train')
prepare_dataset(dataset=test, dataset_type='test')

train
Dealing with missing values, outliers, categorical features...
Dataset is successfully prepared and saved to dataset/, run time (dealing with bad values): 0:00:27.865651
test
Dealing with missing values, outliers, categorical features...
Dataset is successfully prepared and saved to dataset/, run time (dealing with bad values): 0:00:02.552020


In [14]:
train_new = pd.read_csv('dataset/dataset_train.csv', sep=';')
# test_new = pd.read_csv('dataset/dataset_test.csv', sep=';')

In [15]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469475 entries, 0 to 469474
Data columns (total 62 columns):
user_id                    469475 non-null object
is_churned                 469475 non-null int64
level                      469475 non-null int64
donate_total               469475 non-null float64
trans_amt_1                469475 non-null float64
trans_amt_2                469475 non-null float64
trans_amt_3                469475 non-null float64
trans_amt_4                469475 non-null float64
pay_amt_1                  469475 non-null float64
pay_amt_2                  469475 non-null float64
pay_amt_3                  469475 non-null float64
pay_amt_4                  469475 non-null float64
reports_amt_1              469475 non-null float64
reports_amt_2              469475 non-null float64
reports_amt_3              469475 non-null float64
reports_amt_4              469475 non-null float64
sess_with_abusers_amt_1    469475 non-null float64
sess_with_abusers_amt_2    

In [16]:
train_new.describe()

,is_churned,level,donate_total,trans_amt_1,trans_amt_2,trans_amt_3,trans_amt_4,pay_amt_1,pay_amt_2,pay_amt_3,...,gold_spent_1,gold_spent_2,gold_spent_3,gold_spent_4,age,gender,days_between_reg_fl,days_between_fl_df,has_return_date,has_phone_number
count,469475.000000,469475.000000,4.694750e+05,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,...,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000,469475.000000
mean,0.029448,26.792698,4.800206e+04,0.198513,0.182546,0.173069,0.173995,140.415895,134.262708,129.077459,...,73.334719,72.931138,70.150866,69.289718,26.002226,0.933899,14.340597,218.709164,0.882946,0.830589
std,0.169058,12.680296,8.576742e+04,0.573882,0.567848,0.562879,0.553329,597.573949,603.785240,613.631584,...,380.674537,385.894325,407.242002,385.043766,8.341266,0.248460,114.818230,363.410345,0.321485,0.375115
min,0.000000,10.000000,6.000000e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000
25%,0.000000,17.000000,6.312000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,20.000000,1.000000,0.000000,8.000000,1.000000,1.000000
50%,0.000000,23.000000,2.019600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,0.000000,56.000000,1.000000,1.000000
75%,0.000000,36.000000,5.543250e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,31.000000,1.000000,0.000000,266.000000,1.000000,1.000000
max,1.000000,50.000000,4.356043e+06,18.000000,44.285714,26.857143,18.000000,56700.000000,56598.857143,46400.000000,...,36115.000000,47376.857143,62268.000000,56068.000000,80.000000,1.000000,2534.000000,2683.000000,1.000000,1.000000


In [17]:
train_new['is_churned'].value_counts()

0    455650
1     13825
Name: is_churned, dtype: int64

### Балансировка классов

Датасет успешно обработан, пропуски заполнены, явные выбросы обработаны. Однако, в обучающей выборке видим, что присутствует сильный дизбаланс классов - __3/97__. Сбалансируем классы с помощью библиотеки __imblearn__.

В данной библиотеки, как и в целом в балансировке блассов, есть две основные категории методов: __Under-sampling__ и __Over-sampling__. Для "сглаживания" дизбаланса между классами самой интуитивной мерой было бы сделать овер-сэмплинг минорного класса, это можно было бы сделать методом, например, ADASYN (см. [документацию](https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.over_sampling)), но что если одновременно с нехваткой объектов минорного класса в датасете присутствует некоторое количество "излишних" объектов мажорного класса? То есть имеются объекты, у которых вектора признаков очень схожи между собой. Значит, можно убрать часть таких объектов и не потерять в "информативности" выборки. Для решения этой задачи есть ряд методов, например, ClusterCentroids, при котором кластеры схожих объектов в многомерном пространстве объектов признаков заменяются одним синтетическим объектом - центроидом кластера (см. [документацию](https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.under_sampling)).

Для того, чтобы одновременно убрать потенциальные "излишки" в мажорном классе и одновременно увеличить размер минорного класса, можно использовать комбинации under- и over-sampling методов. Для этого есть соответствующие методы __combine.SMOTEENN__ и __combine.SMOTETomek__ (см. [документацию](https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.combine)).

Применим SMOTE для нашей задачи и посмотрим, как изменится баланс классов. SMOTE использует Евклидово расстояние, как метрику близости объектов, для создания синтетических объектов минорного класса, поэтому перед его применением следует отмасштабировать признаки.

In [18]:
#!pip install imblearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

In [19]:
X_train = train_new.drop(['user_id', 'is_churned'], axis=1)
y_train = train_new['is_churned']

X_train_mm = MinMaxScaler().fit_transform(X_train)

In [20]:
%%time
X_train_balanced, y_train_balanced = SMOTE(random_state=42, ratio=0.3). \
                                        fit_sample(X_train_mm, y_train.values)

/home/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


CPU times: user 35.4 s, sys: 2.03 s, total: 37.4 s
Wall time: 9.4 s


Сравним баланс до и после:

In [21]:
from collections import Counter

print('До:', Counter(y_train.values))
print('После:', Counter(y_train_balanced))

До: Counter({0: 455650, 1: 13825})
После: Counter({0: 455650, 1: 136695})


Как и ожидали, баланс выровнялся до соотношения 23/77 (=0.3), как мы и указали в параметрах SMOTE (ratio=0.3).

Балансировка классов позволяет осуществлять __trade-off__ между precision и recall: если нужно максимизировать полноту, а целевой класс минорный, то делаем over-sampling до соотношения классов 1:1 или даже больше, чтобы модель "видела" много объектов целевого класса и хорошо обучилась на них, но в ущерб precision, так как вместе с recall будет расти количество ошибок I рода.

Обычно для максимизации F1 достаточно лишь немного восполнить баланс классов до соотношения __20%-80%__.

При балансировке классов важно помнить, что балансировать нужно только train в обучающей выборке, а test в обучающей выборке оставить без изменений, тогда валидация будет корректной.

### Литература

1. [Понимаем декораторы в Python'e, шаг за шагом. Шаг 1](https://habr.com/ru/post/141411/)
2. [Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/)
3. [Decorators in Python](https://www.geeksforgeeks.org/decorators-in-python/)
4. [Apache Spark™](https://spark.apache.org/)
5. [imbalanced-learn API](https://imbalanced-learn.readthedocs.io/en/stable/api.html)

### Д/З

1. Напишите функцию, которая возвращает сумму двух вещественных аргументов a и b, а к ней декоратор, который делает так, чтобы возвращаемое значение функцией было по модулю 5.
2. Попробуйте описать своими словами основные отличия SMOTE от ADASYN, ознакомившись с документацией к ним.